In [1]:
import os
import gc

import pandas as pd
import numpy as np

import torch

from modelscope import snapshot_download

from transformers import AutoTokenizer, PreTrainedTokenizerFast, LlamaTokenizerFast
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

from tqdm import tqdm

from custom_tokenizers import YueTokenizer

2024-03-31 19:16:25,564 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-03-31 19:16:25,566 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-03-31 19:16:25,615 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 b5a2c5fe01f7460b3e700a8ce7e6fc94 and a total number of 972 components indexed
/root/miniconda3/envs/trans/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PROJ_DIRECTORY = r'AIST4010-Cantonese-Translator/'
DATA_DIRECTORY = r'AIST4010-Cantonese-Translator-Data/'

yue_path = r'AIST4010-Cantonese-Translator/yue_tokenizer/yue_tokenizer.json'
model_path =r'01ai/Yi-6B-Chat'

In [3]:
model_dir = snapshot_download('01ai/Yi-6B-Chat', cache_dir='/root/autodl-tmp', revision='master')
yi_tokenizer = LlamaTokenizerFast.from_pretrained(model_path, use_fast=True, padding_side='right', max_length=512, return_tensors='pt')
yue_tokenizer = PreTrainedTokenizerFast(tokenizer_file=yue_path, padding_side='right', max_length=512, return_tensors='pt')

In [4]:
print(yue_tokenizer.tokenize("嗌 呃 畀 啲 嘢 噃"))
print(yi_tokenizer.tokenize("嗌 呃 畀 啲 嘢 噃"))
print(yue_tokenizer.tokenize("嗌呃畀啲嘢噃"))
print(yi_tokenizer.tokenize("嗌呃畀啲嘢噃"))

#compare ids 
print(yue_tokenizer.convert_tokens_to_ids(yue_tokenizer.tokenize("嗌 呃 畀 啲 嘢 噃")))
print(yue_tokenizer.convert_tokens_to_ids(yue_tokenizer.tokenize("嗌呃畀啲嘢噃")))

['嗌', '呃', '畀', '啲', '嘢', '噃']
['▁', '<0xE5>', '<0x97>', '<0x8C>', '▁', '<0xE5>', '<0x91>', '<0x83>', '▁', '<0xE7>', '<0x95>', '<0x80>', '▁', '<0xE5>', '<0x95>', '<0xB2>', '▁', '<0xE5>', '<0x98>', '<0xA2>', '▁', '<0xE5>', '<0x99>', '<0x83>']
['嗌', '呃', '畀', '啲嘢', '噃']
['▁', '<0xE5>', '<0x97>', '<0x8C>', '<0xE5>', '<0x91>', '<0x83>', '<0xE7>', '<0x95>', '<0x80>', '<0xE5>', '<0x95>', '<0xB2>', '<0xE5>', '<0x98>', '<0xA2>', '<0xE5>', '<0x99>', '<0x83>']
[4391, 4127, 8637, 4324, 4452, 4476]
[4391, 4127, 8637, 19174, 4476]


In [5]:
# add vocab in yue tokenizer but not in yi tokenizer to yi tokenizer
yue_vocab = list(yue_tokenizer.get_vocab().keys())
print(yue_vocab)
new_vocab = set(yue_vocab) - set(yi_tokenizer.get_vocab().keys())
print(len(new_vocab))

print(new_vocab)


['羯', 'ל', '頀', '貯', '抱', '嘅一個市鎮', '屯門', 'ု', '血', '浯', '屋邨', 'I', '匱', '蓍', '當中', '美國嘅', '东', '恵', '첫', '釴', '殍', '𢃕', 'nd', '喺19', '燮', '𩕺', '變得', '留低', '烡', '驟', '䷭', '議席', '完全冇', '碪', '大街', '戰略', 'ⱋ', '配合', '婊', '李', '輸', '昍', '摌', '旎', '蠀', '鋹', '鸓', '袼', '至今', '不得', '德里', '涔', '瀅', '禹', '葆', '💔', '檗', '爖', '圢', '東涌', '二世', '奸', '炠', '水平', '仟', '糶', 'Road', '深', '蔑', '汀', '䷍', 'ܚ', '㇏', '亅', '墈', '<|endoftext|>', '惰', '湎', '憙', '筰', '錸', '笌', '鑴', '規劃', '鸡', '𠼰', '䶩', '籌', 'ー', 'Č', '－', '譚', '猁', '矹', '西部', '痣', '愬', 'ene', '臂', 'শ', '厳', '˘', '啖', '眩', '過程', '獅', '足球代表隊出場', '槃', '燶', '第7', '虜', '匸', '實', '岢', '劳', '敆', '欿', '跏', '鉳', '𦘒', '䀊', '升', '🔚', '好快', '但喺', '䲓', '蟑', '毐', '位置喺阿列', '啵', '蒡', '邱', '騂', '鳽', 'ark', 'ﻍ', '迴', '㜺', '滕', '卅', '崗', '旭', '㦬', '掊', '嘢', '勻', '教會', 'ㅁ', '嘰', '愧', '摤', '芣', '☑', '込', 'Ⱥ', '菜', '鬴', '開胃', '曠', 'ۍ', '键', '齢', '黎講', '亦喺', '复', '⁵', '根', '鏃', '때', '夺', '患', '徵', '髁', '6', 'ᄇ', '庒', '伋', '柧', '罌', '作', '胰', '𢃴', 'um', '機械', '胅', '蚂', 'T

In [6]:
yi_tokenizer.add_tokens(list(new_vocab))

13967

In [7]:
def pre_tokenize(text):
    tokens = yue_tokenizer.tokenize(text)
    return " ".join(tokens)

text = "嗌呃畀啲嘢噃"
pre_tokenized_text = pre_tokenize(text)
print(yi_tokenizer.tokenize(pre_tokenized_text))

['▁嗌', '▁呃', '▁畀', '▁啲嘢', '▁噃']


In [10]:
from transformers import LlamaTokenizer, PreTrainedTokenizerFast

class CustomTokenizer(LlamaTokenizer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.yue_tokenizer = PreTrainedTokenizerFast(tokenizer_file=yue_path, padding_side='right', max_length=512, return_tensors='pt')

    def _tokenize(self, text, **kwargs):
        # Pre-tokenize the text using the yue_tokenizer
        pre_tokenized_text = " ".join(self.yue_tokenizer.tokenize(text))
        
        # Tokenize the pre-tokenized text using the yi_tokenizer
        return super()._tokenize(pre_tokenized_text, **kwargs)

# Load the custom tokenizer
yi_tokenizer = CustomTokenizer.from_pretrained(model_path, padding_side='right', max_length=512, return_tensors='pt')

# Add vocab in yue tokenizer but not in yi tokenizer to yi tokenizer
yue_vocab = list(yue_tokenizer.get_vocab().keys())
new_vocab = set(yue_vocab) - set(yi_tokenizer.get_vocab().keys())
yi_tokenizer.add_tokens(list(new_vocab))

# Test the custom tokenizer
text = "Hello 嗌呃畀啲嘢噃"
print(yi_tokenizer.tokenize(text))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'CustomTokenizer'.


['H', '▁el', '▁lo', '嗌', '呃', '畀', '啲嘢', '噃']


In [11]:
print(yue_tokenizer.tokenize("嗌 呃 畀 啲 嘢 噃"))
print(yi_tokenizer.tokenize("嗌 呃 畀 啲 嘢 噃"))
print(yue_tokenizer.tokenize("嗌呃畀啲嘢噃"))
print(yi_tokenizer.tokenize("嗌呃畀啲嘢噃"))

['嗌', '呃', '畀', '啲', '嘢', '噃']
['嗌', '呃', '畀', '啲', '嘢', '噃']
['嗌', '呃', '畀', '啲嘢', '噃']
['嗌', '呃', '畀', '啲嘢', '噃']


In [ ]:
yi_tokenizer.save_pretrained('/root/new_tokenizer')

('/root/new_tokenizer/tokenizer_config.json',
 '/root/new_tokenizer/special_tokens_map.json',
 '/root/new_tokenizer/tokenizer.model',
 '/root/new_tokenizer/added_tokens.json',
 '/root/new_tokenizer/tokenizer.json')

In [ ]:
print(len(yi_tokenizer.get_vocab()))
print(yi_tokenizer.get_vocab())

77969
{'▁chiral': 19941, '㇏': 74888, '▁medal': 28080, '▁Cards': 30444, '▁BIOS': 37453, '▁descriptions': 20841, '▁entrepreneurial': 49781, '▁fain': 52159, '旃': 66729, 'itars': 38083, '批': 60437, '矋': 69194, '手续': 14732, '▁Desk': 51408, 'огра': 26278, '▁six': 3389, 'quiry': 17761, 'nForm': 21724, '雖然': 23406, '机电': 39974, '䏈': 68925, 'destination': 44469, 'Mac': 15035, 'ꞗ': 69411, '▁consistency': 15881, '▁releg': 44238, 'idual': 44981, '▁shall': 4658, '▁lease': 21605, '▁Territory': 57138, 'intendo': 19444, '▁outs': 17473, 'belie': 20267, '些': 59831, '▁turbulent': 31327, 'ymptotic': 52702, '潾': 67941, '畤': 74795, '燬': 71763, '▁ч': 18925, '▁wasting': 29359, '▁notably': 25693, 'alia': 36650, '黧': 66145, '鑕': 73611, 'tC': 24920, '▁suitable': 9043, '撟': 66124, 'dl': 15196, '▁upstream': 34545, '曾經': 51779, '▁quests': 48340, 'nH': 1266, 'ж': 60367, '▁toug': 23285, '▁conjug': 22810, '理科': 43468, '问': 59870, '6號': 74037, '花园': 19167, 'ﺶ': 64156, '乏': 61206, '話要': 73435, 'plitude': 41006, 'вод': 3